In [1]:
%cd ..

/misc/vlgscratch4/LakeGroup/wentao/multimodal-baby


In [2]:
import itertools
import functools
from collections import namedtuple
import re
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.multimodal_data_module import PAD_TOKEN_ID, UNK_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID
from ngram import NGramModel
from multimodal.utils import map_structure
from analysis_tools.processing import examples_from_dataloader, get_pos_tags, get_word_pos_cnt, get_word_pos_stat_from_word_pos_cnt, get_pos_stats_for_words, get_model_losses_on_data_batches
from analysis_tools.sumdata import *
from analysis_tools.build_data import build_data
from analysis_tools.pos_tags import *
from analysis_tools.utils import *
from analysis_tools.word_categories import *


figsize = (13, 12)
sns.set_theme(
    style='white',
    font='monospace',
    rc={
        'figure.figsize': figsize,
        'axes.grid': False,
        'xtick.bottom': False,
        'xtick.top': False,
        'ytick.left': False,
        'ytick.right': False,
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
    }
)

np.set_printoptions(suppress=True, precision=2, linewidth=120)
pd.options.display.width = 120

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos_mapping = pos_mappings[2][1]

used_poses = ["noun", "verb", "adjective", "adverb", "function word", "cardinal number", "."][:2]

In [3]:
def pos_tagged_seq_repr(tokens, pos_tags):
    return ' '.join(f'{token}/{pos}' for token, pos in zip(tokens, pos_tags))


class Template(namedtuple('TemplateTuple', ['seq', 'pos', 'idx'])):
    """A template where the token at idx of seq is to be filled.
    """

    def __hash__(self):
        return hash((tuple(self.seq[:self.idx]), tuple(self.seq[self.idx+1:])))

    def __str__(self):
        return ' '.join(
            '_' * len(token) if i == self.idx else token
            for i, (token_id, token) in enumerate(
                (token_id, idx2word[token_id]) for token_id in self.seq)
            if token_id not in [SOS_TOKEN_ID, EOS_TOKEN_ID])


def template_from_str(s, use_unk=False):
    """Construct Template from str s. This is for manual construction of templates.
    Inputs:
        s: str. A sentence that is tokenized (tokens are separated by space) and containing exactly one slot, represented by '_'.
            Example: "here 's a _ and a kitty ."
    Return:
        a Template.
    """
    tokens = s.split()
    token_ids = []
    idx = None
    for i, token in enumerate(tokens):
        if re.fullmatch(r'_+', token):
            if idx is not None:
                raise Exception(f'more than one slot in template: {s}')
            idx = i
            token_id = UNK_TOKEN_ID
        else:
            try:
                token_id = word2idx[token]
            except KeyError:
                if use_unk:
                    token_id = UNK_TOKEN_ID
                else:
                    raise
        token_ids.append(token_id)
    if idx is None:
        raise Exception(f'must have one slot in template: {s}')

    token_ids = [SOS_TOKEN_ID] + token_ids + [EOS_TOKEN_ID]
    idx += 1
    pos = ["."] * len(token_ids)
    return Template(token_ids, pos, idx)


def templates_from_example(token_ids, pos_tags):
    for i, (token_id, pos) in enumerate(zip(token_ids, pos_tags)):
        if pos_mapping[pos] in used_poses:
            yield Template(token_ids, pos_tags, i)


def templates_from_examples(examples, pos_tags, print_pos_tagged_seq=False):
    tot, cnt_filtered = 0, 0

    for example, y_pos_tags in zip(examples, pos_tags):
        x, y, y_len, raw_y = example
        tot += 1
        if print_pos_tagged_seq:
            print(pos_tagged_seq_repr(raw_y[0].split(), y_pos_tags[1:]))
        cnt_present_tokens = sum((pos_mapping[pos] not in ["."] for pos in y_pos_tags))
        if cnt_present_tokens <= 2:
            continue
        y_len = y_len.item()
        y = y[:y_len].tolist()
        if UNK_TOKEN_ID in y:
            continue

        cnt_filtered += 1
        yield example, templates_from_example(y, y_pos_tags)

    print(f'filtered {cnt_filtered} / {tot} = {cnt_filtered / tot :.2%} examples')


def joined_templates_from_examples(*args, **kwargs):
    for example, example_templates in templates_from_examples(*args, **kwargs):
        for template in example_templates:
            yield example, template


def construct_batch_by_filling_template(x, template, word_ids):
    batch_size = len(word_ids)
    x_batch = x.unsqueeze(0).expand(batch_size, *([-1] * x.dim()))
    y = torch.tensor(template.seq)
    y_batch = y.unsqueeze(0).repeat(batch_size, *([1] * y.dim()))
    y_batch[:, template.idx] = torch.tensor(word_ids)
    y_len = torch.tensor(len(template.seq))
    y_len_batch = y_len.unsqueeze(0).expand(batch_size, *([-1] * y_len.dim()))
    raw_y_batch = [[]] * batch_size
    return x_batch, y_batch, y_len_batch, raw_y_batch


def run_model_on_template(model, example, template, word_ids, batch_size=256):
    """Run model and get the whole sentence losses on template filled with every word_id in word_ids
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        batch_size: the batch size
    Returns:
        losses: an np.ndarray of length len(word_ids)
    """

    x, y, y_len, raw_y = example
    x = x.to(device=device)

    batches = (construct_batch_by_filling_template(x, template, word_ids[i : i + batch_size])
               for i in range(0, len(word_ids), batch_size))
    return get_model_losses_on_data_batches(model, batches, use_tqdm=False)


def find_index(lst, e):
    for i, x in enumerate(lst):
        if x == e:
            return i
    return None


def get_prob_dist(model, example, template, word_ids, word_cats, pos_mapping=identity, batch_size=256, top_k=None):
    """Run model on template and get the probability distribution of categories;
    check whether model gives higher probability to the correct category.
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        word_cats: np.ndarray of same length as word_ids; the categories of each word in word_ids
        pos_mapping: callable, used to map pos to cat
        batch_size: the batch size used to run the model
        top_k: print top k predictions along with the ground-truth label; None if unwanted
    Returns:
        pd.Series with categories indexing the probability distribution
    """

    losses = run_model_on_template(model, example, template, word_ids, batch_size=batch_size)
    probs = F.softmax(-losses, 0)

    gt_word_id = template.seq[template.idx]
    gt_word_pos = template.pos[template.idx]
    gt_word_cat = pos_mapping(gt_word_pos)
    gt_word_idx = find_index(word_ids, gt_word_id)
    if gt_word_idx is None:  # the ground-truth word is not in the vocab of prediction
        print(f'ground-truth word {idx2word[gt_word_id]} not in the vocab of prediction')
        # append the ground-truth word to the vocab
        gt_word_idx = len(word_ids)
        word_ids = np.append(word_ids, gt_word_id)
        word_cats = np.append(word_cats, gt_word_cat)
        losses = torch.cat([losses, torch.tensor([np.inf], dtype=losses.dtype, device=losses.device)])
        probs = torch.cat([probs, torch.tensor([0.], dtype=probs.dtype, device=probs.device)])
    else:
        # set the category of the ground-truth word to the category according to the template
        if word_cats[gt_word_idx] != gt_word_cat:
            word_cats = np.copy(word_cats)
            word_cats[gt_word_idx] = gt_word_cat

    if top_k is not None:
        words = idx2word[word_ids]
        print_losses = False
        if print_losses:
            print_top_values(-losses, words, labels=torch.tensor(gt_word_idx), value_formatter=lambda loss: f'{-loss:6.3f}')
        print_top_values(probs, words, labels=torch.tensor(gt_word_idx), value_formatter=prob_formatter)

    probs_ser = pd.Series(probs.cpu())
    return probs_ser.groupby(word_cats).sum()

In [4]:
# build data and vocab according to the model
data, args = build_data(return_args=True)
dataset_name = args.dataset
vocab = data.datasets['train'].vocab
vocab_size = len(vocab)
print(f'{vocab_size = }')
word2idx = vocab
idx2word = [None] * vocab_size
for word, idx in word2idx.items():
    idx2word[idx] = word
idx2word = np.array(idx2word)

my_batch_size = 256
dataloader_fns = {
    'train': lambda: data.train_dataloader(batch_size=my_batch_size, shuffle=False, drop_last=False),
    'val': lambda: data.val_dataloader(batch_size=my_batch_size)[0],
    'test': lambda: data.test_dataloader(batch_size=my_batch_size)[0],
}

checkpoint_path = "checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt"
print(f"load model from {checkpoint_path}")
model = MultiModalLitModel.load_from_checkpoint(checkpoint_path, map_location=device)
model.to(device)
model.eval()

# get the pos tags of all words in vocab
train_dataloader_fn = dataloader_fns['train']
train_pos_tags = get_pos_tags(train_dataloader_fn(), dataset_name, 'train')
word_pos_cnt = get_word_pos_cnt(train_dataloader_fn(), train_pos_tags)
word_pos_stat = get_word_pos_stat_from_word_pos_cnt(word_pos_cnt)
pos_stats = get_pos_stats_for_words(idx2word, word_pos_stat, pos_mapping=pos_mapping.get)
idx2pos2 = np.array([pos_stat[0][0] if pos_stat else '.' for pos_stat in pos_stats])

for pos in sorted(set(pos_mapping.values())):
    pos_word_ids = np.nonzero(idx2pos2 == pos)[0]
    print(f'#{pos:15s}: {len(pos_word_ids)}')

idx2used = np.isin(idx2pos2, used_poses)

for token_id in [PAD_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID, UNK_TOKEN_ID]:
    idx2used[token_id] = False

for idx, (word, pos_stat) in enumerate(zip(idx2word, pos_stats)):
    used_pos_stat = [(pos, cnt) for pos, cnt in pos_stat if pos in used_poses]
    if not used_pos_stat:
        continue
    word_cnt = sum(cnt for pos, cnt in used_pos_stat)
    stat_str = ' '.join(f'{pos:15} {cnt:5} {cnt / word_cnt :6.1%}' for pos, cnt in used_pos_stat)
    print(f'{word:10s} {stat_str}')

    # remove too ambiguous words
    if used_pos_stat[0][1] / word_cnt < 0.9:
        idx2used[idx] = False

word_ids = np.nonzero(idx2used)[0]
word_cats = idx2pos2[word_ids]
used_vocab_size = len(word_ids)
print(f'{used_vocab_size = }')

/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Using base transforms
Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created. Skipping this step.
Shuffled training metadata file has already been created. Skipping this step.
Evaluation frames have already been filtered. Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Filtered evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Vocabulary file already exists. Skipp

In [5]:
# observation: many ambiguous cases seem to be confusing nouns vs V-ings/passive verbs.


splits = ['val']

my_untypical_words = ' '.join(
    possessives + negations + be_verbs + do_verbs + modal_verbs +
    pronoun_contractions + other_contractions + quantifiers + pos_ambiguous_words +
    special_tokens
).split()
my_untypical_word_ids = [word2idx[word] for word in my_untypical_words if word in word2idx][:0]
filter_out_gt_word_ids = [SOS_TOKEN_ID, EOS_TOKEN_ID]

seen_templates = set()
for split in ['train']:
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_dataloader(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    seen_templates.update(template for example, template in joined_templates_from_examples(examples, pos_tags))

for split in splits:
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_dataloader(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    golds, preds, corrects = [], [], []
    for example, template in joined_templates_from_examples(examples, pos_tags):
        gt_word_id = template.seq[template.idx]
        if gt_word_id in filter_out_gt_word_ids:
            continue
        if not idx2used[gt_word_id]:
            continue
        print(template)
        if template in seen_templates:
            print('seen template; skipped')
            continue

        gold = pos_mapping[template.pos[template.idx]]
        prob_dist = get_prob_dist(model, example, template, word_ids, word_cats, pos_mapping=pos_mapping.get, top_k=5)
        print(prob_dist)
        argmax_idx = prob_dist.argmax()
        pred = prob_dist.index[argmax_idx]
        correct = pred == gold
        print(f'{gold = }, {pred = }, {correct = }')
        golds.append(gold)
        preds.append(pred)
        corrects.append(correct)

    print(f'label distribution: ')
    frac_format = lambda a, b, prec=2: f'{a} / {b} = {a/b:.{prec}%}'
    for pos in used_poses:
        cnt = sum(gold == pos for gold in golds)
        print(f'{pos}: {frac_format(cnt, len(golds))}')
    print(f'accuracy: {frac_format(sum(corrects), len(corrects))}')

load cached pos tags: dataset_cache/saycam/train.pos.cache
filtered 20379 / 33737 = 60.41% examples
load cached pos tags: dataset_cache/saycam/val.pos.cache
we should ____ on some lights , huh ?
  5.2% turn     |  91.2% put        5.2% turn       0.4% leave      0.4% keep       0.4% get     
noun    0.001679
verb    0.998321
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we should turn on some ______ , huh ?
 14.0% lights   |  14.0% lights    13.4% toys       9.5% water      7.6% music      5.4% books   
noun    0.988316
verb    0.011684
dtype: float32
gold = 'noun', pred = 'noun', correct = True
here , ___ me show you .
 99.2% let      |  99.2% let        0.4% watch      0.1% lets       0.1% give       0.0% tell    
noun    0.000437
verb    0.999563
dtype: float32
gold = 'verb', pred = 'verb', correct = True
here , let me ____ you .
 30.9% show     |  30.9% show      23.1% get       20.6% see        2.7% carry      2.5% find    
noun    0.012404
verb    0.987596
dtype: fl

  0.2% keep     |  76.9% put       16.5% putting    1.6% push       0.7% get        0.5% getting 
noun    0.002049
verb    0.997951
dtype: float32
gold = 'verb', pred = 'verb', correct = True
oh whoops you want to be very careful keep it in the ____ .
  5.3% bowl     |  14.9% bin       13.8% mirror     9.6% pantry     7.0% bathtub    5.3% bowl    
noun    0.984215
verb    0.015785
dtype: float32
gold = 'noun', pred = 'noun', correct = True
i __ wearing black shoes today .
 62.4% 'm       |  62.4% 'm        30.7% am         1.7% was        1.0% 've        1.0% have    
noun    0.000803
verb    0.999197
dtype: float32
gold = 'verb', pred = 'verb', correct = True
i 'm _______ black shoes today .
 70.5% wearing  |  70.5% wearing    6.8% wear       5.4% making     2.9% seen       1.7% doing   
noun    0.015177
verb    0.984823
dtype: float32
gold = 'verb', pred = 'verb', correct = True
i 'm wearing black _____ today .
  7.3% shoes    |  25.1% socks     16.3% shirt      7.5% one        7.3% 

  0.2% grandpa  |  14.7% sam        7.7% mommy      7.5% lamb       5.9% lid        5.8% bunny   
noun    0.906826
verb    0.093174
dtype: float32
gold = 'noun', pred = 'noun', correct = True
where 'd grandpa __ ?
 97.5% go       |  97.5% go         0.7% are        0.7% do         0.1% is         0.1% say     
noun    0.000628
verb    0.999372
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you _____ to look at the-
  1.1% going    |  90.6% want       4.5% have       1.3% need       1.1% going      0.4% like    
noun    0.000416
verb    0.999584
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you going to ____ at the-
 32.3% look     |  32.3% look       5.4% be         4.9% press      4.3% draw       4.2% looking 
noun    0.126376
verb    0.873624
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you going to look at ____
  0.1% the-     |  12.6% daycare   11.2% marmite    5.5% everything   4.6% sam        3.2% kitty   
noun    0.94166
verb    0.05834


 67.7% is       |  67.7% is        23.1% 's         4.6% s          1.8% was        0.6% are     
noun    0.001253
verb    0.998747
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you ____ sitting here , do n't you ?
 11.2% like     |  45.4% 're       21.2% are       11.2% like       8.5% try        4.2% were    
noun    0.001582
verb    0.998418
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you like _______ here , do n't you ?
  0.4% sitting  |  18.9% looking    7.9% play       4.2% throwing   4.0% eating     3.7% things  
noun    0.30498
verb    0.69502
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you like sitting here , __ n't you ?
 91.2% do       |  91.2% do         4.9% are        2.7% did        0.6% were       0.2% have    
noun    0.000112
verb    0.999888
dtype: float32
gold = 'verb', pred = 'verb', correct = True
if you ___ happy and you know it , and you really want to show it , and if you 're happy and you
 83.0% 're      |  83.0% '

 22.1% come     |  30.7% hang      22.1% come      10.8% hold       6.9% go         6.4% put     
noun    0.017826
verb    0.982173
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you sure like _____ , do n't you .
  0.8% bunny    |   8.7% bread      4.1% looking    3.5% nappies    3.3% everything   3.3% babies  
noun    0.770362
verb    0.229638
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you sure like bunny , __ n't you .
 96.4% do       |  96.4% do         1.9% are        1.4% did        0.1% were       0.1% have    
noun    0.000442
verb    0.999558
dtype: float32
gold = 'verb', pred = 'verb', correct = True
yeah , can you ___ that back please ?
 38.2% put      |  38.2% put       24.8% give      15.6% throw      6.8% bring      4.1% get     
noun    0.005751
verb    0.994249
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there we __ , in and out .
 98.4% go       |  98.4% go         0.7% are        0.3% have       0.1% sit        0.1% put   

...

 96.4% like     |  96.4% like       1.5% are        0.5% 're        0.5% want       0.2% try     
noun    0.000182
verb    0.999818
dtype: float32
gold = 'verb', pred = 'verb', correct = True
mmm , you like _______ yourself do n't you ?
 43.4% feeding  |  43.4% feeding   15.3% watching  11.4% looking    1.7% pressing   1.5% throwing
noun    0.149735
verb    0.850265
dtype: float32
gold = 'verb', pred = 'verb', correct = True
mmm , you like feeding yourself __ n't you ?
 96.2% do       |  96.2% do         2.5% did        0.7% are        0.3% does       0.1% were    
noun    0.000078
verb    0.999922
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there ___ snakes , and and bunny .
 33.9% are      |  55.2% is        33.9% are        9.9% 's         0.3% 're        0.2% s       
noun    0.001278
verb    0.998722
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there are ______ , and and bunny .
  0.7% snakes   |  21.4% fish      10.1% blocks     8.4% bunnies    7.4%

 95.3% want     |  95.3% want       1.9% going      1.0% need       0.6% have       0.3% like    
noun    0.000262
verb    0.999738
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to _____ a flower ?
  0.1% water    |  33.8% see       14.3% draw       9.1% try        8.3% have       5.2% get     
noun    0.007034
verb    0.992966
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to water a ______ ?
  1.3% flower   |  27.7% bottle    24.1% bit        6.4% banana     3.1% ball       2.6% picture 
noun    0.99027
verb    0.00973
dtype: float32
gold = 'noun', pred = 'noun', correct = True
he ___ boots on .
 50.4% has      |  50.4% has       19.7% 's         6.3% is         4.4% got        3.2% have    
noun    0.002438
verb    0.997562
dtype: float32
gold = 'verb', pred = 'verb', correct = True
he has _____ on .
  0.1% boots    |  24.7% pants     16.5% shoes      9.9% shorts     9.6% socks      6.1% buttons 
noun    0.866053
verb    0.133947
dtype: f

 78.8% going    |  78.8% going     18.4% go         0.7% listening   0.2% giving     0.2% have    
noun    0.003474
verb    0.996526
dtype: float32
gold = 'verb', pred = 'verb', correct = True
bit out of order , but -- and it 's a different because we 're not going to the _____ .
 44.8% store    |  44.8% store     11.7% sandpit   10.8% beach      5.9% eggs       5.7% library 
noun    0.991674
verb    0.008327
dtype: float32
gold = 'noun', pred = 'noun', correct = True
to ____ it open you have to push this in
 54.0% make     |  54.0% make      12.1% open      10.0% push       4.6% get        2.5% watch   
noun    0.006819
verb    0.993181
dtype: float32
gold = 'verb', pred = 'verb', correct = True
to make it open you ____ to push this in
 49.0% have     |  49.0% have      27.7% want       9.8% need       1.7% put        1.5% go      
noun    0.005802
verb    0.994199
dtype: float32
gold = 'verb', pred = 'verb', correct = True
to make it open you have to ____ this in
  4.4% push     |  8

noun    0.001094
verb    0.998906
dtype: float32
gold = 'verb', pred = 'verb', correct = True
and here 's this ___ .
 63.9% one      |  63.9% one        8.7% ball       6.2% baby       3.6% book       1.4% sock    
noun    0.993556
verb    0.006444
dtype: float32
gold = 'noun', pred = 'noun', correct = True
we ___ not going to open it either .
 68.2% 're      |  68.2% 're       18.7% are        3.4% have       3.2% ’re        1.9% 's      
noun    0.000388
verb    0.999612
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we 're not _____ to open it either .
 98.3% going    |  98.3% going      0.6% supposed   0.5% have       0.2% trying     0.1% need    
noun    0.000707
verb    0.999293
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we 're not going to ____ it either .
  0.0% open     |  31.3% do        13.5% have       7.2% make       6.3% carry      4.2% eat     
noun    0.05102
verb    0.94898
dtype: float32
gold = 'verb', pred = 'verb', correct = True
can i 

  3.0% mouth    |  60.1% foot       6.5% hand       3.0% mouth      2.1% water      2.0% coming  
noun    0.959390
verb    0.040609
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you ____ to put those in the bin ?
 84.2% want     |  84.2% want      10.3% going      2.0% need       1.8% have       0.4% like    
noun    0.000294
verb    0.999706
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to ___ those in the bin ?
 88.2% put      |  88.2% put        2.5% throw      1.8% get        1.6% bring      1.0% see     
noun    0.000345
verb    0.999655
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to put those in the ___ ?
 20.2% bin      |  20.2% bin       17.7% bag        9.4% cup        6.9% box        6.4% backpack
noun    0.996563
verb    0.003437
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you ___ standing on a stick
 24.9% 're      |  63.3% are       24.9% 're        5.7% re         2.6% were       0.8% have    
n

In [6]:
# TODO: examine whether BabyBERTa vocab is similar to SAYCam's; if true, we can try that dataset.


manual_templates = [
    "here 's _ .",
    "here 's a _ .",
    "here 's the _ .",
    "here 're _ .",
    "an _ .",
    "a _ .",
    "a kitty and a _ .",
    "a _ and a kitty .",
    "kitties and _ .",
    "_ and kitties .",
    "a doggy and a _ .",
    "a _ and a doggy .",
    "a cow and a _ .",
    "a _ and a cow .",
    "cows and _ .",
    "_ and cows .",
    "a train and a _ .",
    "a _ and a train .",
    "a car and a _ .",
    "a _ and a car .",
    "a banana and a _ .",
    "an apple and a _ .",
    "shoes and _ .",
    "_ and shoes .",
    "a jacket and a _ .",
    "a baby and a _ .",
    "a _ and a baby .",
    "have the _ .",
    "have some _ .",
    "take the _ .",
    "eat _ .",
    "eat the _ .",
    "drink _ .",
    "drink some _ .",
    "wear the _ .",
    "go to _ .",
    "go to the _ .",
    "get on the _ .",
    "get into the _ .",
    "the girl the boys love _ there .",
    "the girl that the boys see _ there .",
    "the girl that the boys saw _ there .",
    "the girl that the boys love _ there .",
    "the girl that the boys loved _ there .",
    "the boys the girl likes _ there .",
    "the boys the girl liked _ there .",
    "the boys the girl loved _ there .",
    "the boys that the girl saw _ there .",
    "the horse ran past the _ .",
    "the horse ran past the barn _ .",
    "the horse ran past the barn _",
    "the doggy ran past the _ .",
    "the birdie flew past the _ .",
]

x = torch.zeros_like(next(examples_from_dataloader(dataloader_fns['val']()))[0])

for template_str, template in zip(manual_templates, map(template_from_str, manual_templates)):
    print(template)
    y = torch.tensor(template.seq)
    y_len = torch.tensor(len(template.seq))
    example = x, y, y_len, [template_str]
    prob_dist = get_prob_dist(model, example, template, word_ids, word_cats, pos_mapping=pos_mapping.get, top_k=5)
    print(prob_dist)

here 's _____ .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  38.1% one        5.2% bunny      3.4% grandpa    2.5% bread      2.3% mommy   
.       0.000000
noun    0.970923
verb    0.029077
dtype: float32
here 's a _____ .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  22.6% ball       7.0% kitty      6.5% train      3.8% banana     3.3% doggy   
.       0.000000
noun    0.995478
verb    0.004522
dtype: float32
here 's the _____ .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  30.5% ball       5.2% kitty      3.5% train      2.3% egg        2.2% siren   
.       0.000000
noun    0.993968
verb    0.006032
dtype: float32
here 're _____ .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  11.1% one       10.3% done       7.8% fish       4.2% sam        2.3% water   
.       0.000000
noun    0.808057
verb    0.191943
dtype: float32
an _____ .
ground-truth word <unk> not in the vocab o

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  47.3% was        7.0% stay       6.3% is         5.2% play       4.1% looking 
.       0.000000
noun    0.071937
verb    0.928063
dtype: float32
the boys the girl likes _____ there .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  11.2% come       6.0% stay       5.2% moved      4.2% look       4.1% are     
.       0.000000
noun    0.359253
verb    0.640747
dtype: float32
the boys the girl liked _____ there .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  17.4% come       9.9% yesterday   5.9% play       5.6% stay       5.3% look    
.       0.000000
noun    0.413247
verb    0.586753
dtype: float32
the boys the girl loved _____ there .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  10.9% was       10.6% play       5.9% come       5.8% stay       5.5% juice   
.       0.000000
noun    0.237896
verb    0.762104
dtype: float32
the boys 